In [66]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device used: {device.type}')

Device used: cpu


# Hyperparameters for project

In [67]:
train_size = 0.7
test_size = 1 - train_size
num_epochs = 5
batch_size = 4
learning_rate = 0.001

# Data preprocessing


## Import the dataset
Dataset should be imported as a pytorch dataloader for batch optimization

Custom Dataset

## **Data augmentation**

To augment the data before training, we will attempt to use two methods:


1.   Scaling
> Scaling is used because we wish to taken into account the varying structure size of tumors and skulls in the images.

2.   Noise Injection
> Noise injection is used to help assist the model in learning the complex patterns around the tumors and make it more robust to small changes in the data.
> We will experiment with both Gaussian (random), and salt-and-paper (random values to min. or max. values, 0 to 255) noise injection.

All combinations of these will be used to determine their effectiveness, and if they introduce any *bias*, *artifacts*, or *overfitting* both in isolation, or combination.

The order of the data augmentation will be:
1.   No Data Augmentation
2.   Scaling
3.   Noise Injection
4.   Scaling, Noise Injection

In [68]:
from torchvision.transforms import v2

def add_noise_gaussian(tensor, mean = 0, std = 0.05):
    """
    Parameters:
    - tensor: PyTorch tensor data type without noise (input)
    - mean: Mean of the Gaussian distribution
    - std: Standard deviation of the Gaussian distribution

    Returns:
    - tensor + noise: PyTorch tensor data type with noise (output)
    """
    noise = torch.randn(tensor.size()) * std + mean
    return tensor + noise

def add_noise_salt_pepper(tensor, salt_prob = 0.02, pepper_prob = 0.02):
    """
    Parameters:
    - tensor: PyTorch tensor data type without noise (input)
    - salt_prob: Probability that salt noise is added (full white)
    - pepper_prob: Probability that pepper noise is added (full black)

    Returns:
    - tensor + salt_mask = pepper_mask: PyTorch tensor data type with noise (output)
    that ensured to be between 0 and 1
    """

    salt_mask = (torch.rand_like(tensor) < salt_prob).float()
    pepper_mask = (torch.rand_like(tensor) < pepper_prob).float()

    return torch.clamp((tensor + salt_mask - pepper_mask), 0, 1)

# Abitrary values (set to double of base image height*width)
resize_x = 256
resize_y = 256

# Set resize_x and resize_y before using these transforms
# All transforms convert it to a tensor with the dimensions of (Channels, Height, Width)
transforms = {
    'none': v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True)
                        ]),
    'scale': v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Resize((resize_x, resize_y), antialias=True)
                        ]),
    'noise_gaussian': v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Lambda(lambda x: add_noise_gaussian(x))
                        ]),
    'noise_salt_pepper': v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Lambda(lambda x: add_noise_salt_pepper(x))
                        ]),
    'all_gaussian':     v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Resize((resize_x, resize_y), antialias=True),
                        v2.Lambda(lambda x: add_noise_gaussian(x))
                        ]),
    'all_salt_pepper':  v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Resize((resize_x, resize_y), antialias=True),
                        v2.Lambda(lambda x: add_noise_salt_pepper(x))
                        ])
}

In [69]:
from torch.utils.data import Dataset
from torchvision.transforms import v2
from PIL import Image
import os

class CustomDataset(Dataset):

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.class_folders = os.listdir(root_dir)
        self.class_to_idx = {class_folder: i for i, class_folder in enumerate(self.class_folders)}
        self.images = self.make_dataset()

    def make_dataset(self):
        images = []
        for class_folder in self.class_folders:
            class_path = os.path.join(self.root_dir, class_folder)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                images.append((img_path, self.class_to_idx[class_folder]))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path)

        random_number = self.random_gen.uniform(0, 1)

        if self.transform:
            image = self.apply_random_transform(image, random_number)

        return image, label
    
    def apply_random_transform(self, image, random_number):
        # Example: Apply different transformations based on the random number
        if random_number < 0.25:
            return transforms['noise_gaussian'](image)
        elif random_number < 0.5:
            return transforms['noise_salt_pepper'](image)
        else:
            return transforms['none'](image)
    

In [70]:
from torch.utils.data import DataLoader

transform = v2.Compose([
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True)
])

dataset = CustomDataset(root_dir='Dataset', transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

## Data normalization

# Architecture of the network

**CNN Model Creation**

In [71]:
class CustomCNN(nn.Module):
    def __init__(self, weights="DEFAULT"):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  # MRI images are grayscale, so in_channels=1
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)  # Adjust for the flattened conv3 output
        self.fc2 = nn.Linear(256, 4)  # 4 classes in our dataset

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 8 * 8)  # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x